# Part III

### Create a Travel Itinerary with a Corresponding Map

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
# Import the API key.
from config2 import g_key

In [2]:
city_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
0,Sulam,IL,32.61,35.33,75.99,83,9,1.99,clear sky,0.00,0
1,Namatanai,PG,-3.67,152.43,86.16,65,4,5.82,clear sky,0.00,0
2,Bluff,NZ,-46.60,168.33,48.27,79,95,16.93,light rain,0.23,0
3,Santa Cruz,BO,-17.80,-63.17,59.00,93,90,1.01,overcast clouds,0.00,0
4,San Patricio,MX,19.22,-104.70,80.24,80,100,6.11,light rain,0.19,0


In [3]:
# 4. Prompt the customer to answer cities they want to go
city_1= str(input('Where do you want to visit for the first choice?'))
city_2= str(input('Where do you want to visit for the second choice?'))
city_3= str(input('Where do you want to visit for the third choice?'))
city_4= str(input('Where do you want to visit for the fourth choice?'))

Where do you want to visit for the first choice?Keansburg
Where do you want to visit for the second choice?Sterling
Where do you want to visit for the third choice?Mayo
Where do you want to visit for the fourth choice?Fredericksburg


In [4]:
# 5. Create separate DataFrames for each city on the travel route, using the loc attribute.
city_1 = city_data_df.loc[(city_data_df["City"] == city_1)]
city_1

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
363,Keansburg,US,40.44,-74.13,75.0,87,7,8.37,clear sky,0.0,0


In [5]:
city_2 = city_data_df.loc[(city_data_df["City"] == city_2)]
city_2

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
286,Sterling,US,39.01,-77.43,78.01,83,75,5.82,broken clouds,0.0,0


In [6]:
city_3 = city_data_df.loc[(city_data_df["City"] == city_3)]
city_3

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
117,Mayo,US,38.89,-76.51,78.01,94,1,6.53,clear sky,0.0,0


In [7]:
city_4 = city_data_df.loc[(city_data_df["City"] == city_4)]
city_4

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
147,Fredericksburg,US,38.3,-77.46,78.8,83,1,4.43,clear sky,0.0,0


In [8]:
four_cities = pd.concat([city_1, city_2, city_3, city_4])
four_cities

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour)
363,Keansburg,US,40.44,-74.13,75.00,87,7,8.37,clear sky,0.0,0
286,Sterling,US,39.01,-77.43,78.01,83,75,5.82,broken clouds,0.0,0
117,Mayo,US,38.89,-76.51,78.01,94,1,6.53,clear sky,0.0,0
147,Fredericksburg,US,38.30,-77.46,78.80,83,1,4.43,clear sky,0.0,0


In [9]:
four_cities2 = four_cities[['City','Lat','Lng']]
four_cities2

,City,Lat,Lng
363,Keansburg,40.44,-74.13
286,Sterling,39.01,-77.43
117,Mayo,38.89,-76.51
147,Fredericksburg,38.30,-77.46


In [10]:
# 6. To map each city on the route, get the latitude-longitude pairs for each city DataFrame as tuples.
# 7. Set travel mode 
import gmaps
import gmaps.datasets
gmaps.configure(api_key=g_key)

# Latitude-longitude pairs
Keansburg = (40.44, -74.13)
Sterling = (39.01, -77.43)
Mayo = (38.89, -76.51)
Fredericksburg = (38.30, -77.46)

fig = gmaps.figure()
four_cities_direction = gmaps.directions_layer(Keansburg, Sterling,waypoints=[Mayo, Fredericksburg], travel_mode='DRIVING')
fig.add_layer(four_cities_direction)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in four_cities.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        four_cities.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descriptions</dt><dd>{Current Descriptions}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in four_cities.iterrows()]

In [13]:
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Holiday Inn Hazlet</dd>\n<dt>City</dt><dd>Keansburg</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Descriptions</dt><dd>clear sky</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hampton Inn & Suites Washington-Dulles International Airport</dd>\n<dt>City</dt><dd>Sterling</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Descriptions</dt><dd>broken clouds</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Resorts Resource Group International</dd>\n<dt>City</dt><dd>Mayo</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Descriptions</dt><dd>clear sky</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Courtyard by Marriott Fredericksburg Historic District</dd>\n<dt>City</dt><dd>Fredericksburg</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Descriptions</dt><dd>clear sky</dd>\n</dl>\n']

In [14]:
four_cities

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions,Rain (inches last hour),Snow (inches last hour),Hotel Name
363,Keansburg,US,40.44,-74.13,75.00,87,7,8.37,clear sky,0.0,0,Holiday Inn Hazlet
286,Sterling,US,39.01,-77.43,78.01,83,75,5.82,broken clouds,0.0,0,Hampton Inn & Suites Washington-Dulles Interna...
117,Mayo,US,38.89,-76.51,78.01,94,1,6.53,clear sky,0.0,0,Resorts Resource Group International
147,Fredericksburg,US,38.30,-77.46,78.80,83,1,4.43,clear sky,0.0,0,Courtyard by Marriott Fredericksburg Historic ...


In [19]:
# 10. On the marker layer map, make sure each city has a pop-up marker contains Hotel Name, City, Country, Current Descriptions
locations = four_cities[["Lat", "Lng"]]
fig1 = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig1.add_layer(four_cities_direction)
fig1.add_layer(marker_layer)

# Call the figure to plot the data.
fig1

Figure(layout=FigureLayout(height='420px'))